## Introduction
#### Categorizing a dataset and applying a function to each group (aggregation or transformation) is a critical component os data analysis workflow.
#### After 'loading', 'merging' and 'preparing' a dataset, you may need to compute 'group statistics' or 'pivot' tables for reporting or visualization purposes.
#### A reason why SQL is so popular is because of the ease with which data can be joined, filtered, transformed and agregated. But they are somewhat contrained in group operations.
#### With the expresiveness of Python and pandas we can perform complex group operations by using functions that accept pandas object or NumPy array.
#### NOTE - Aggregation of time series data is special case of 'groupby', also refered to as 'resampling'. 

## GroupBy Mechanics
#### The 'split-apply-combine' term describes group operations succinctly.
#### In the first stage, data in pandas object is 'split' into groups based on one or more keys. Splitting is performed on particular axis of the object.
#### Next, a function is 'applied' to each group producing a new value.
#### Finally, the results of those applications are 'combined' into a result object. The form of the result will depend on what is being done to the data.

#### Grouping key can take many forms. Keys can be of different types:
####     1. A list of values having same length as the axis being grouped.
####     2. A value indicating column name in a DataFrame.
####     3. A dict or Series providing correspondence between values on the axis being grouped an group names.
####     4. A function to be invoked on axis index or individual labels in the index.
#### Note that last three methods are actually shortcuts for producing array of values to split up the object.

In [8]:
import pandas as pd
import numpy as np

df = pd.DataFrame({'key1': ['a', 'a', 'b', 'b', 'a'],
                  'key2': ['one', 'two', 'one', 'two', 'one'],
                  'data1': np.random.randn(5),
                  'data2': np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,1.033605,-0.208085
1,a,two,-0.414086,0.026467
2,b,one,-0.590407,0.812162
3,b,two,0.057372,0.365072
4,a,one,-0.511034,2.224576


#### Computing mean of the data1 column of above DataFrame object using labels from key2 can be done in a number of ways.
#### One way is to access data1 and call 'groupby' with the column (Series) at key1. This will return a GroupBy object.
#### The object has not computed anything except for intermediate data about the group key 'key1'. It has all the information to apply some operation to each of the groups. eg - 'mean' method.
#### Important thing to note is that the Series has been aggregated by a group key, producing a new Series indexed by unique values in key1 column. Hence the result index also has the name 'key1'.

In [10]:
grouped = df['data1'].groupby(df['key1'])
grouped

In [11]:
grouped.mean()

key1
a    0.036162
b   -0.266518
Name: data1, dtype: float64

#### If we had passed multiple arrays in the groupby column as list, we get a resulting Series with hierarchical index.
#### This index consists of unique arrangement of keys passed in the groupby method.
#### The group keys can be any arrays of the right length.

In [13]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one     0.261286
      two    -0.414086
b     one    -0.590407
      two     0.057372
Name: data1, dtype: float64

In [14]:
means.unstack()

key2,one,two
key1,,
a,0.261286,-0.414086
b,-0.590407,0.057372


In [15]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])
df['data1'].groupby([states, years]).mean()

California  2005   -0.414086
            2006   -0.590407
Ohio        2005    0.545488
            2006   -0.511034
Name: data1, dtype: float64

#### Mostly the grouping information would be present in the same data you want to work on. In this case, you can pass column names as group keys.
#### If you have a non numeric data in a column, it is called a 'nuisance column' and it is excluded from the aggregation result. But we can use these columns for grouping.
#### By default, all numeric columns are aggregated. Although it is possible to filter down to a subset.
#### Regardless of objective, a useful group method is 'size', which return a Series having group sizes. Missing values in a group key are excluded from the result.

In [17]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,0.036162,0.680986
b,-0.266518,0.588617


In [18]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one   0.261286  1.008245
     two  -0.414086  0.026467
b    one  -0.590407  0.812162
     two   0.057372  0.365072

In [19]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

### Iterating Over Groups
#### GroupBy objects support iteration, generating sequence of 2-tuples having group name with chunk of data.
#### In case of multiple keys, first element in the tuple will be tuple of keys.

In [21]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one  1.033605 -0.208085
1    a  two -0.414086  0.026467
4    a  one -0.511034  2.224576
b
  key1 key2     data1     data2
2    b  one -0.590407  0.812162
3    b  two  0.057372  0.365072


In [22]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one  1.033605 -0.208085
4    a  one -0.511034  2.224576
('a', 'two')
  key1 key2     data1     data2
1    a  two -0.414086  0.026467
('b', 'one')
  key1 key2     data1     data2
2    b  one -0.590407  0.812162
('b', 'two')
  key1 key2     data1     data2
3    b  two  0.057372  0.365072


#### We can do whatever we want with the chunks of data. A useful thing would be to compute a dict of the data chunks as a one-liner.
#### By default 'groupby' groups on axis=0, but you can group on any other axes. 
#### We can print out the groups too.

In [24]:
pieces = dict(list(df.groupby('key1')))
pieces

{'a':   key1 key2     data1     data2
 0    a  one  1.033605 -0.208085
 1    a  two -0.414086  0.026467
 4    a  one -0.511034  2.224576, 'b':   key1 key2     data1     data2
 2    b  one -0.590407  0.812162
 3    b  two  0.057372  0.365072}

In [25]:
pieces['b']

,key1,key2,data1,data2
2,b,one,-0.590407,0.812162
3,b,two,0.057372,0.365072


In [27]:
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [28]:
grouped = df.groupby(df.dtypes, axis=1)

for dtype, group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0  1.033605 -0.208085
1 -0.414086  0.026467
2 -0.590407  0.812162
3  0.057372  0.365072
4 -0.511034  2.224576
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


### Selecting a Column or Subset of Columns
#### Indexing a GroupBy object created from a DataFrame with column name or array of column names has same effect as column subsetting for aggregation.
#### What it means is that doing this:

In [30]:
'''
df.groupby('key1')['data1']
df.groupby('key1')[['data2']]
'''

"\ndf.groupby('key1')['data1']\ndf.groupby('key1')[['data2']]\n"

#### is the syntactic sugar (i.e. has the same effect) for:

In [32]:
'''
df['data1'].groupby(df['key1'])
df[['data2']].groupby(df['key1'])
'''

"\ndf['data1'].groupby(df['key1'])\ndf[['data2']].groupby(df['key1'])\n"

#### For large datasets, it is desirable if we aggregate using only a few columns.
#### The object returned by this indexing is a grouped DataFrame if a list or array is passed or a grouped Series if a single column is passed as scalar.

In [34]:
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one   1.008245
     two   0.026467
b    one   0.812162
     two   0.365072

In [35]:
s_grouped = df.groupby(['key1', 'key2'])['data2']
s_grouped

In [36]:
s_grouped.mean()

key1  key2
a     one     1.008245
      two     0.026467
b     one     0.812162
      two     0.365072
Name: data2, dtype: float64

### Grouping with Dicts and Series
#### Grouping information may be present in forms other than an array. Instead os passing an array, we can also pass a dict to groupby. Using unused grouping keys is OK.
#### We can also use Series in groupby, which can be viewed as fixed-sized mapping.

In [38]:
people = pd.DataFrame(np.random.randn(5,5),
                     columns = ['a', 'b', 'c', 'd', 'e'],
                     index = ['Joe', 'steve', 'Wes', 'Jim', 'Travis'])
people.iloc[2:3, [1,2]] = np.nan # a few NA values

people

,a,b,c,d,e
Joe,0.446110,-0.242829,-0.625072,-1.009086,-0.439756
steve,0.138213,0.433581,-0.246506,0.017845,-0.937134
Wes,1.832978,NaN,NaN,0.340975,-2.868593
Jim,-1.802749,0.348062,-1.539908,0.668386,0.346682
Travis,0.140066,-1.241122,-2.421142,0.286051,0.797616


In [39]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
          'd': 'blue', 'e': 'red', 'f': 'orange'}

In [40]:
by_column = people.groupby(mapping, axis=1)
by_column.sum()

,blue,red
Joe,-1.634158,-0.236475
steve,-0.228661,-0.365340
Wes,0.340975,-1.035615
Jim,-0.871522,-1.108005
Travis,-2.135091,-0.303440


In [41]:
map_series = pd.Series(mapping)

map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [42]:
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


### Grouping with functions
#### Functions is a more generic way of defining a group mapping. Every function passed as group key will be called once per index value, with th return values used as group names.
#### Mixing functions with arrays, dicts or Series is not a problem as everything gets converted to arrays internally.

In [44]:
people.groupby(len).sum()

,a,b,c,d,e
3,0.476339,0.105232,-2.164979,0.000274,-2.961667
5,0.138213,0.433581,-0.246506,0.017845,-0.937134
6,0.140066,-1.241122,-2.421142,0.286051,0.797616


In [45]:
key_list = ['one', 'one', 'one', 'two', 'two']

people.groupby([len, key_list]).min()

a         b         c         d         e
3 one  0.446110 -0.242829 -0.625072 -1.009086 -2.868593
  two -1.802749  0.348062 -1.539908  0.668386  0.346682
5 one  0.138213  0.433581 -0.246506  0.017845 -0.937134
6 two  0.140066 -1.241122 -2.421142  0.286051  0.797616

### Grouping by Index Levels
#### Another convinience for hierarchically indexed datasets is the ability to group bu one of the index levels.
#### To group by a certain level, pass the level number or name with the 'level' keyword.

In [47]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                    [1, 3, 5, 1, 3]],
                                   names=['cty', 'tenor'])
hier_df = pd.DataFrame(np.random.randn(4,5), columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0     -0.294187 -1.912265  0.805537  1.857416 -0.511496
1      0.460410 -0.876451 -0.180717 -0.676693 -1.873716
2     -1.881580  0.207316 -1.165858 -2.042299  0.846380
3      0.391582 -0.171104 -0.560497  1.325864  1.096357

In [48]:
hier_df.groupby(level='cty', axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


### Data Aggregation
#### Aggregation refers to data transformation that produces scalar values from arrays. rg - mean, count, min, sum, etc.
#### You can use aggregation created by you or call any method defined on the grouped object.
#### Certain functions are not defined for groupby, but can be used with it. groupby slices up the object into pieces, calls the function for each piece and then assembles them into result object.

In [50]:
df

,key1,key2,data1,data2
0,a,one,1.033605,-0.208085
1,a,two,-0.414086,0.026467
2,b,one,-0.590407,0.812162
3,b,two,0.057372,0.365072
4,a,one,-0.511034,2.224576


In [51]:
grouped = df.groupby('key1')
grouped['data1'].quantile(0.9)

key1
a    0.744067
b   -0.007406
Name: data1, dtype: float64

#### To use your own aggregation functions, pass it to the 'aggregate' or 'agg' method.
#### Some methods, though not strictly aggregation functions, also work with agg.
#### NOTE - Custom aggregation functions are much slower than optimized functions. There is some overhead (eg - function calls, data rearrangement) in constructing the intermediate group chunks.

In [53]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,1.544638,2.432661
b,0.647779,0.447090


In [54]:
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0  0.036162  0.865170 -0.511034 -0.462560 -0.414086  0.309759   
b      2.0 -0.266518  0.458049 -0.590407 -0.428462 -0.266518 -0.104573   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     1.033605   3.0  0.680986  1.341922 -0.208085 -0.090809  0.026467   
b     0.057372   2.0  0.588617  0.316140  0.365072  0.476845  0.588617   

                          
           75%       max  
key1                      
a     1.125521  2.224576  
b     0.700390  0.812162

### Column-Wise and Multiple Function Application
#### Aggregating a Series or columns of a DataFrame is matter of using 'aggregate' with desired function or calling a method eg - mean or std.
#### There might be a need to aggregate using a different function based on the column, or multiple functions at once.

In [57]:
tips = pd.read_csv('examples/tips.csv')
tips['tip_pct'] = tips['tip'] / tips['total_bill']

tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


#### For descriptive statistics, you can pass the name of function as a string.
#### We can also pass a list of functions or function names. You will get back a DataFrame with column names from the functions.

In [59]:
grouped = tips.groupby(['day', 'smoker'])
grouped_pct = grouped['tip_pct']
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [60]:
grouped_pct.agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

#### But you do not need to accept the names the groupby gives to the columns.
#### lambda functions have the name "< lambda >" which is difficult to identify (you can check the function's __name__ attribute).
#### If you pass a list of (name, function) tuples, first element of each tuple will used as DataFrame column names.                                                                          

In [63]:
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

#### You have more options with DataFrame. You can specify a list of functions to apply to all the columns or different functions per column.
#### The resulting DataFrame will have hierarchical columns. Similar to aggregating each column seperately using 'concat' to glue results with column names in the 'keys' argument.

In [65]:
functions = ['count', 'mean', 'max']
result = grouped['tip_pct', 'total_bill'].agg(functions)
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [66]:
result['tip_pct']

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

#### We can also use list of tuples to have custom names for result columns, just like in Series.
#### You can also apply different functions to one or more columns bu passing a dict containing a mapping of column names with function specifications to 'agg'.
#### A DataFrame will have hierarchical columns only if multiple functions are applied to atleast one column.

In [68]:
ftuples = [('Durchschnitt', 'mean'), ('Abweichung', np.var)]

grouped['tip_pct', 'total_bill'].agg(ftuples)

tip_pct              total_bill            
            Durchschnitt Abweichung Durchschnitt  Abweichung
day  smoker                                                 
Fri  No         0.151650   0.000791    18.420000   25.596333
     Yes        0.174783   0.002631    16.813333   82.562438
Sat  No         0.158048   0.001581    19.661778   79.908965
     Yes        0.147906   0.003767    21.276667  101.387535
Sun  No         0.160113   0.001793    20.506667   66.099980
     Yes        0.187250   0.023757    24.120000  109.046044
Thur No         0.160298   0.001503    17.113111   59.625081
     Yes        0.163863   0.001551    19.190588   69.808518

In [69]:
grouped.agg({'tip': np.max, 'size': 'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [70]:
grouped.agg({'tip_pct': ['min', 'max', 'mean', 'std'],
            'size': 'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

### Returning Aggregated Data without Row Indexes
#### Usually aggregated data comes back with an index (mostly hierarchical), composed of unique group key combinations.
#### This is not always desirable. we can disable this by passing 'as_index=False' to 'groupby'.
#### We can also use 'reset_index' on the result to get same format, but the former method avoids unnecessary computations.

In [72]:
tips.groupby(['day', 'smoker'], as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


### Apply: General split-apply-combine
#### The most general-purpose groupby method is 'apply'. It splits and object into pieces, invokes passed function on each piece and then concatenates the pieces together.
#### when we use groupby with a function, the function is called on each row of the DataFrame and the results are glued together using 'pandas.concat', labeling the pieces with group names. The result has a hierarchical index whose inner level has index values from original DataFrame.

In [74]:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]


top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [75]:
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

#### If you want to pass a function to apply with other arguments or keywords, you can pass them after the function.
#### NOTE - Beside basic usage, it requires creativity to use apply and get the most out of it. What happens inside a function depends on you, as long as the function returns a pandas object or scalar value.

In [77]:
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

#### Inside groupby, when you want to invoke a method, there is a shortcut for it.

In [79]:
# Normal route
result = tips.groupby('smoker')['tip_pct'].describe()

result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [80]:
result.unstack('smoker')

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

In [81]:
# Shortcut

f = lambda x: x.describe()
grouped.apply(f)

total_bill        tip       size    tip_pct
day  smoker                                                   
Fri  No     count    4.000000   4.000000   4.000000   4.000000
            mean    18.420000   2.812500   2.250000   0.151650
            std      5.059282   0.898494   0.500000   0.028123
            min     12.460000   1.500000   2.000000   0.120385
            25%     15.100000   2.625000   2.000000   0.137239
            50%     19.235000   3.125000   2.000000   0.149241
            75%     22.555000   3.312500   2.250000   0.163652
            max     22.750000   3.500000   3.000000   0.187735
     Yes    count   15.000000  15.000000  15.000000  15.000000
            mean    16.813333   2.714000   2.066667   0.174783
            std      9.086388   1.077668   0.593617   0.051293
            min      5.750000   1.000000   1.000000   0.103555
            25%     11.690000   1.960000   2.000000   0.133739
            50%     13.420000   2.500000   2.000000   0.173913
            75%     18.665000   3.240000   2.000000   0.209240
            max     40.170000   4.730000   4.000000   0.263480
Sat  No     count   45.000000  45.000000  45.000000  45.000000
            mean    19.661778   3.102889   2.555556   0.158048
            std      8.939181   1.642088   0.784960   0.039767
            min      7.250000   1.000000   1.000000   0.056797
            25%     14.730000   2.010000   2.000000   0.136240
            50%     17.820000   2.750000   2.000000   0.150152
            75%     20.650000   3.390000   3.000000   0.183915
            max     48.330000   9.000000   4.000000   0.291990
     Yes    count   42.000000  42.000000  42.000000  42.000000
            mean    21.276667   2.875476   2.476190   0.147906
            std     10.069138   1.630580   0.862161   0.061375
            min      3.070000   1.000000   1.000000   0.035638
            25%     13.405000   2.000000   2.000000   0.091797
            50%     20.390000   2.690000   2.000000   0.153624
...                       ...        ...        ...        ...
Sun  No     std      8.130189   1.224785   1.032674   0.042347
            min      8.770000   1.010000   2.000000   0.059447
            25%     14.780000   2.000000   2.000000   0.139780
            50%     18.430000   3.020000   3.000000   0.161665
            75%     25.000000   3.920000   4.000000   0.185185
            max     48.170000   6.000000   6.000000   0.252672
     Yes    count   19.000000  19.000000  19.000000  19.000000
            mean    24.120000   3.516842   2.578947   0.187250
            std     10.442511   1.261151   0.901591   0.154134
            min      7.250000   1.500000   2.000000   0.065660
            25%     17.165000   3.000000   2.000000   0.097723
            50%     23.100000   3.500000   2.000000   0.138122
            75%     32.375000   4.000000   3.000000   0.215325
            max     45.350000   6.500000   5.000000   0.710345
Thur No     count   45.000000  45.000000  45.000000  45.000000
            mean    17.113111   2.673778   2.488889   0.160298
            std      7.721728   1.282964   1.179796   0.038774
            min      7.510000   1.250000   1.000000   0.072961
            25%     11.690000   1.800000   2.000000   0.137741
            50%     15.950000   2.180000   2.000000   0.153492
            75%     20.270000   3.000000   2.000000   0.184843
            max     41.190000   6.700000   6.000000   0.266312
     Yes    count   17.000000  17.000000  17.000000  17.000000
            mean    19.190588   3.030000   2.352941   0.163863
            std      8.355149   1.113491   0.701888   0.039389
            min     10.340000   2.000000   2.000000   0.090014
            25%     13.510000   2.000000   2.000000   0.148038
            50%     16.470000   2.560000   2.000000   0.153846
            75%     19.810000   4.000000   2.000000   0.194837
            max     43.110000   5.000000   4.000000   0.241255

[64 rows x 4 columns]

### Suppressing the Group Keys
#### The result from groupby has been an object with hierarchical index fromed by group keys along with indexes of each piece of original object.
#### We can disable this by passing 'group_keys=False' to groupby.

In [83]:
tips.groupby('smoker', group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


### Quantile and Bucket Analysis
#### pandas has some tools for slicing data into buckets with bins of your choosing or sample quantiles. eg - 'cut' and 'qcut'.
#### Combining these functions with groupby makes it convinient to eprform bucket or quantile analysis on a dataset.

In [86]:
frame = pd.DataFrame({'data1': np.random.randn(1000),
                     'data2': np.random.randn(1000)})

quartiles = pd.cut(frame.data1, 4)
quartiles[:10]

0     (-3.283, -1.668]
1     (-0.0593, 1.549]
2    (-1.668, -0.0593]
3     (-0.0593, 1.549]
4    (-1.668, -0.0593]
5    (-1.668, -0.0593]
6     (-0.0593, 1.549]
7     (-0.0593, 1.549]
8     (-0.0593, 1.549]
9    (-1.668, -0.0593]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-3.283, -1.668] < (-1.668, -0.0593] < (-0.0593, 1.549] < (1.549, 3.158]]

#### The categorical object returned by cut can be passed directly to groupby to compute set of statistics for any column.
#### For equal-sized buckets based on sample quantiles, we can use 'qcut'. We can pass 'labels=False' to get quantile numbers.

In [89]:
def get_stats(group):
    return {'min': group.min(), 'max': group.max(),
           'count': group.count(), 'mean': group.mean()}

grouped = frame.data2.groupby(quartiles)
grouped.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
"(-3.283, -1.668]",52.0,3.164256,0.128052,-2.280419
"(-1.668, -0.0593]",457.0,3.152965,-0.044853,-3.462515
"(-0.0593, 1.549]",433.0,3.836951,-0.066945,-3.237170
"(1.549, 3.158]",58.0,2.245450,0.111512,-1.798201


In [90]:
grouping = pd.qcut(frame.data1, 10, labels=False)
grouped = frame.data2.groupby(grouping)

grouped.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
0,100.0,3.164256,0.043477,-2.496022
1,100.0,2.135533,-0.170752,-3.016819
2,100.0,2.979919,0.106324,-3.462515
3,100.0,1.873858,-0.126911,-2.408925
4,100.0,2.533027,-0.005920,-2.369702
5,100.0,3.836951,0.059023,-3.237170
6,100.0,2.740522,-0.206760,-3.154231
7,100.0,2.742002,-0.023638,-2.201357
8,100.0,2.309644,-0.017650,-2.028481


### Example: Filling Missing Values with Group-Specific Values
#### When cleaning up missing data, you can drop the missing columns or rows using 'dropna'. But you may also want to impute (fill in) the null values using a fixed or derived value using 'fillna'.
#### To fill a value varying by group, you can group the data and use 'apply' with a function on each of the data chunk.

In [92]:
s = pd.Series(np.random.randn(6))
s[::2] = np.nan
s

0         NaN
1    0.245974
2         NaN
3   -0.617814
4         NaN
5    0.616995
dtype: float64

In [93]:
s.fillna(s.mean())

0    0.081718
1    0.245974
2    0.081718
3   -0.617814
4    0.081718
5    0.616995
dtype: float64

In [98]:
states = ['Ohio', 'New York', 'Vermont', 'Florida',
         'Oregon', 'Nevada', 'California', 'Idaho']

# Create 4 copies of elements in both East and West as key
group_key = ['East'] * 4 + ['West'] * 4
data = pd.Series(np.random.randn(8), index=states)

data

Ohio          1.891586
New York      0.408207
Vermont       1.604762
Florida       0.036644
Oregon       -0.255685
Nevada        1.689558
California   -0.434870
Idaho        -0.577308
dtype: float64

In [99]:
# setting some values to NA
data[['Vermont', 'Nevada', 'Idaho']] = np.nan
data

Ohio          1.891586
New York      0.408207
Vermont            NaN
Florida       0.036644
Oregon       -0.255685
Nevada             NaN
California   -0.434870
Idaho              NaN
dtype: float64

In [100]:
data.groupby(group_key).mean()

East    0.778812
West   -0.345278
dtype: float64

In [101]:
# setting the NA values with group means
fill_mean = lambda g: g.fillna(g.mean())
data.groupby(group_key).apply(fill_mean)

Ohio          1.891586
New York      0.408207
Vermont       0.778812
Florida       0.036644
Oregon       -0.255685
Nevada       -0.345278
California   -0.434870
Idaho        -0.345278
dtype: float64

#### You may even have predeefined fill values by group. We can use the name attribute in the groups to our advantage.

In [103]:
fill_values = {'East': 0.5, 'West': -1}
fill_func = lambda g: g.fillna(fill_values[g.name])

data.groupby(group_key).apply(fill_func)

Ohio          1.891586
New York      0.408207
Vermont       0.500000
Florida       0.036644
Oregon       -0.255685
Nevada       -1.000000
California   -0.434870
Idaho        -1.000000
dtype: float64

### Example: Random sampling and Permutation
#### If you want to draw a random sample from a large dataset for an application or simulation.
#### There are a number of ways to do it, one of which is to use 'sample' method for Series.

In [105]:
# Hearts, Spades, Clubs, Diamonds
suits = ['H', 'S', 'C', 'D']
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ['A'] + list(range(2, 11)) + ['J', 'K', 'Q']
cards = []

for suit in suits:
    cards.extend(str(num) + suit for num in base_names)
    
deck = pd.Series(card_val, index=cards)

#### We have created a Series whose index contains card names and values used in Blackjack and other games. We have kept ace as 1 for simplicity.
#### We can draw a hand of any number of cards from the deck using 'sample'.
#### We can also get random cards from each suit. We have the suit as the last character of each card, which we can use to group and then use apply for the sample function.
#### To not show the suit, we can pass 'group_keys=False' with the groupby function.

In [107]:
deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

In [108]:
def draw(deck, n=5):
    return deck.sample(n)

draw(deck)

9S     9
8D     8
QS    10
7D     7
5S     5
dtype: int64

In [110]:
# Get last letter for suit
get_suit = lambda card: card[-1]

deck.groupby(get_suit).apply(draw, n=2)

C  4C      4
   KC     10
D  JD     10
   7D      7
H  7H      7
   QH     10
S  9S      9
   10S    10
dtype: int64

In [111]:
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)

AC     1
3C     3
3D     3
KD    10
JH    10
3H     3
JS    10
9S     9
dtype: int64

### Example: Group WeightedAverage and Correlation
#### Using the split-apply-combine paradigm, groupby operates between columns in a DataFrame or two Series. this makes group aggregation methods possible.    

In [113]:
df = pd.DataFrame({'category': ['a', 'a', 'a', 'a',
                               'b', 'b', 'b', 'b'],
                  'data': np.random.randn(8),
                  'weights': np.random.randn(8)})
df

,category,data,weights
0,a,1.638956,0.548527
1,a,-0.516697,0.662717
2,a,0.639473,-1.285644
3,a,-1.341274,1.839168
4,b,0.656749,-0.664400
5,b,1.108143,0.507994
6,b,-0.146337,-0.402249
7,b,2.183043,-0.594460


In [114]:
grouped = df.groupby('category')
get_wavg = lambda g: np.average(g['data'], weights=g['weights'])

grouped.apply(get_wavg)

category
a   -1.548291
b    0.964589
dtype: float64

#### Let's take an example of financial dataset taken from Yahoo! Finance with end-of-day prices with S&P index (SPX) of few stocks.
#### A task of interest is getting yearly correlation of daily return with SPX.

In [116]:
close_px = pd.read_csv('examples/stock_px_2.csv', parse_dates = True,
                      index_col = 0)

close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
AAPL    2214 non-null float64
MSFT    2214 non-null float64
XOM     2214 non-null float64
SPX     2214 non-null float64
dtypes: float64(4)
memory usage: 86.5 KB


In [120]:
close_px[-4:]

,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


#### We can start by creating a function computing pairwise correlation of each column with SPX column.
#### Next we compute the percent change using 'pct_change'. We then group these changes by year, which can be extracted from each row label with a 1 line function.
#### We can also compute inter-column correlations. eg - annual correlations between Apple and Microsoft

In [122]:
spx_corr = lambda x: x.corrwith(x['SPX'])
rets = close_px.pct_change().dropna()
get_year = lambda x: x.year

by_year = rets.groupby(get_year)
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


In [123]:
by_year.apply(lambda g: g['AAPL'].corr(g['MSFT']))

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

### Example: Group-Wise Linear Regression
#### groupby can also be used to perform complex group-wise statistical analysis, as long as the function returns pandas object or scalar value.
#### eg - We can perform Ordinary Least Square (OLS) regression of chunks of data.

In [126]:
import statsmodels.api as sm

def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params

by_year.apply(regress, 'AAPL', ['SPX'])

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514


### Pivot Tables and Cross-Tabulation
#### 'pivot table' is a data summarization tool used in spreadsheets and other data analysis software.
#### It aggregates table by one or more keys, arranging data in a rectangle with some group keys along rows and others along columns.
#### Pivot tables are possible in pandas through 'groupby' combined with 'reshape' utilizing hierarchical indexing.
#### DataFrame also has 'pivot_table' method, and there is also a top-level 'pandas.pivot_table' function.
#### pivot_table also add partial totals, known as 'margins', in addition to the groupby interface.

In [129]:
tips.pivot_table(index=['day', 'smoker'])

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

#### pivot_table allows you to aggregate by certain columns, group by certaing columns with 'index' parameters, and define columns with 'columns' parameter.
#### We could augment this table to include partial totals by passing 'margins=True'. It will add All row and column labels with corresponding values being group statistics for all data within single tier.
#### The 'All' values are aggregations without taking into account the present groups in the columns or rows.

In [131]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'],
                columns='smoker')

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

In [132]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'],
                columns='smoker', margins=True)

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

#### To use a different aggregation function, pass it to 'aggfunc'. eg - 'count' or len will give cross-tabulation (frequencies) of group sizes.
#### If some combinations are empty (NA), you can pass a 'fill_value'.

In [136]:
tips.pivot_table('tip_pct', index=['time', 'smoker'], columns='day', 
                 aggfunc=len, margins=True)

day             Fri   Sat   Sun  Thur    All
time   smoker                               
Dinner No       3.0  45.0  57.0   1.0  106.0
       Yes      9.0  42.0  19.0   NaN   70.0
Lunch  No       1.0   NaN   NaN  44.0   45.0
       Yes      6.0   NaN   NaN  17.0   23.0
All            19.0  87.0  76.0  62.0  244.0

In [138]:
tips.pivot_table('tip_pct', index=['time', 'size', 'smoker'], columns='day', 
                 aggfunc='mean', fill_value=0)

day                      Fri       Sat       Sun      Thur
time   size smoker                                        
Dinner 1    No      0.000000  0.137931  0.000000  0.000000
            Yes     0.000000  0.325733  0.000000  0.000000
       2    No      0.139622  0.162705  0.168859  0.159744
            Yes     0.171297  0.148668  0.207893  0.000000
       3    No      0.000000  0.154661  0.152663  0.000000
            Yes     0.000000  0.144995  0.152660  0.000000
       4    No      0.000000  0.150096  0.148143  0.000000
            Yes     0.117750  0.124515  0.193370  0.000000
       5    No      0.000000  0.000000  0.206928  0.000000
            Yes     0.000000  0.106572  0.065660  0.000000
       6    No      0.000000  0.000000  0.103799  0.000000
Lunch  1    No      0.000000  0.000000  0.000000  0.181728
            Yes     0.223776  0.000000  0.000000  0.000000
       2    No      0.000000  0.000000  0.000000  0.166005
            Yes     0.181969  0.000000  0.000000  0.158843
       3    No      0.187735  0.000000  0.000000  0.084246
            Yes     0.000000  0.000000  0.000000  0.204952
       4    No      0.000000  0.000000  0.000000  0.138919
            Yes     0.000000  0.000000  0.000000  0.155410
       5    No      0.000000  0.000000  0.000000  0.121389
       6    No      0.000000  0.000000  0.000000  0.173706

### Cross-Tabulations: Crosstab
#### Cross-Tabulation (crosstab for short) is a special case of pivot table that computes group frequencies.
#### In some scenarios, 'pandas.crosstab' is a more convenient way to summarize data.
#### The first 2 arguments can be either an array or Series or list of arrays.

In [171]:
d = {0: [1, 'USA', 'Right'],
    1: [2, 'JP', 'Left'],
    2: [3, 'USA', 'Left'],
    3: [4, 'JP', 'Right'],
    4: [5, 'JP', 'Left'],
    5: [6, 'USA', 'Left']}
data = pd.DataFrame.from_dict(d, orient='index',
                              columns=['Sample', 'Nationality', 'Handedness'])
data

,Sample,Nationality,Handedness
0,1,USA,Right
1,2,JP,Left
2,3,USA,Left
3,4,JP,Right
4,5,JP,Left
5,6,USA,Left


In [172]:
pd.crosstab(data.Nationality, data.Handedness, margins=True)

Handedness,Left,Right,All
Nationality,,,
JP,2,1,3
USA,2,1,3
All,4,2,6


In [173]:
pd.crosstab([tips.time, tips.day], tips.smoker, margins=True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244